<a href="https://colab.research.google.com/github/btoneil2021/Search-Minecraft-Wiki/blob/main/MinecraftWikiScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
HOW TO USE:
1. Type API key below
2. Run
'''
api_key = "[INSERT YOUR API KEY HERE]"

# 1 (and 2 I guess). Scrape Data off Minecraft Wiki

In [ ]:
%%shell
pip install chromedriver_autoinstaller
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
import sys
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import threading

In [ ]:
def get_soup(url):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')

  driver = webdriver.Chrome(options=chrome_options)

  driver.get(url)

  driver.implicitly_wait(1)

  soup = BeautifulSoup(driver.page_source, 'html.parser')

  driver.quit()

  return soup

In [ ]:
# 2 Options
# 1. The page is found and is automatically opened
# 2. The page is not found due to some kind of error and the first link in the search is taken
def search_for_page(search_term):
  '''
  input: search_term - (string) what you're looking for
  output: url - (string) the url of the page of results you're looking for
  '''
  # I'm just going to be lazy and add an "i" at the end of everything so everything gets kicked to the search page by default
  url = f'https://minecraft.wiki/w/Special:Search?search={search_term.replace(" ", "+")}i'

  soup = get_soup(url)
  url_addition = soup.find("a", {"data-serp-pos": "0"})["href"]
  url = f'https://minecraft.wiki{url_addition}'

  return url

In [ ]:
def extract_info_box(soup):
  info_box = soup.find("div", {"class": "notaninfobox"})
  info = {}

  # Start by Grabbing all the top-of-table information
  info["name"] = info_box.find("div", {"class": "mcwiki-header infobox-title"}).text.strip()
  info["image_url"] = "https://minecraft.wiki" + info_box.find("div", {"class": "infobox-imagearea animated-container"}).find("img")["src"]

  # Ok now we can grab the info from each table section
  info["details"] = {}
  table = info_box.find("table", {"class": "infobox-rows"})
  table_rows = table.find_all("tr")
  for row in table_rows:
    key = row.find("th")
    if key is None:
      continue
    value = row.find("td")
    if value is None:
      continue
    info["details"][key.text.strip()] = value.text.strip()

  return info

In [ ]:
def extract_main_paragraph(soup):
  main_paragraph = soup.find("div", {"class": "mw-body-content mw-content-ltr"})
  main_paragraph = main_paragraph.find_all(["p", "h2", "h3"])


  tables = soup.find_all("table")
  paragraphs_in_tables = []
  for table in tables:
    paragraphs_in_tables.extend(table.find_all("p"))

  filtered = [el for el in main_paragraph if el.name == 'h2' or el not in paragraphs_in_tables]
  main_paragraph = [el.text.strip() for el in filtered]
  return main_paragraph

# 3. Throw Everything into ChatGPT

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

In [ ]:
def make_chatgpt_request(prompt):
  chat_completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {
              "role": "user",
              "content": prompt
          }
      ]
  )
  return chat_completion.choices[0].message.content

In [ ]:
basic_feature_extraction = '''You are an AI designed to assist with extracting specific features from user requests related to the game Minecraft. Your task is to analyze the user query (delimited by triple backticks) and extract two key pieces of information:

Subject: Identify the main subject of the sentence. This is typically a Minecraft item, entity, block, or structure (full names only).
Summary: Provide a brief summary of the specific information or request that the user wants to know about the subject. Include any information that may be important.
Here are some examples to guide you:

Example 1:
User query: "How do I find diamonds in Minecraft?
Subject: "diamonds"
Summary: "How to find"

Example 2:
User query: "What are the best enchantments for a sword?"
Subject: "sword"
Summary: "best enchantments"

When processing each query, please answer with only the subject and the summary, separated by a backslash.

```

'''

generate_output = '''You are an AI designed to extract specific answers from provided text based on a given summary. Your task is to read the given paragraphs and answer the summary as accurately as possible using the information from the text.
Do not change anything in the summary.

Here is the format you should follow:

Summary: This is a brief statement or question that you need to answer.
Paragraphs: These are the detailed texts from which you will extract the necessary information to answer the summary.
Answer: This is the accurate and concise response to the summary based on the provided paragraphs.
Use the following example as a guide:

Example:
Summary: benefits of a beacon
Paragraphs: Beacons are powerful items in Minecraft that provide various status effects to players within a certain radius. When activated, beacons can grant players speed, haste, resistance, jump boost, or strength. Additionally, a fully powered beacon can give regeneration or a secondary status effect. Beacons are often used in bases to enhance player abilities and provide strategic advantages during combat or resource gathering.
Answer: Beacons provide status effects such as speed, haste, resistance, jump boost, and strength. Fully powered beacons can also give regeneration or a secondary effect, enhancing player abilities and providing strategic advantages in combat and resource gathering.

When processing each query, please answer with only the answer to the summary.

The summary and paragraphs (respectively) are defined below, delimited by triple backticks.
```

'''

In [ ]:
def prompt(prompt):
  summary = make_chatgpt_request(basic_feature_extraction + prompt + "\n```")
  summary = summary.strip("\'\"").split("\n")
  summary = [summary[0].strip("Subject: ").strip('"'),summary[1].strip("Summary: ").strip('"')]

  subject = summary[0]
  summary = summary[1]

  info = [None]
  paragraphs = [None]

  threads = [
      threading.Thread(target=lambda: info.__setitem__(0, extract_info_box(soup))),
      threading.Thread(target=lambda: paragraphs.__setitem__(0, extract_main_paragraph(soup)))
  ]

  url = search_for_page(subject)
  soup = get_soup(url)

  threads[0].start()
  threads[1].start()

  threads[0].join()
  threads[1].join()

  summary = make_chatgpt_request(generate_output + summary + "\n```" + "\n".join(info[0]) + "\n".join(paragraphs[0]))
  return summary

In [ ]:
def main():
  print(prompt(input("What would you like to ask? ")))

In [ ]:
main()

What would you like to ask? What is dirt?
Summary: What is dirt used for in Minecraft?

Answer: Dirt is primarily used for farming in Minecraft, but due to its abundance, it can also be used as a readily available building block.
